


# Gibbs sampling

Originally by Weichao Qiu, modified for Fall 2017 by Donald Li

If you find this ipython notebook is unclear or contains bugs, please contact Chenxi at cxliu@jhu.edu  
If there's an error says "something is undefined", please run the cell that contains the definition or use "menu -> cell -> run all above"

# Foreground/background classification.

Here we consider a model for foreground/background classification that can include spatial context. Intuitively, neighboring pixels in the image are likely to belong to the same class, i.e. are likely to be either all background or all foreground. This is a form of prior knowledge, or natural statistic, which can be learnt by analyzing natural images.

For pixel $i$, the foreground label is $ S_i = 1 $, and background label is $ S_i = -1 $.

The prior term in the energy encourages neighbouring pixels to have the same intensity ($N(i)$ is the set of pixels neighboring $i$):  
$ E_p[S] = \gamma \sum_{i} \sum_{j \in N(i)} { - S_i S_j} $ 

The data term is defined as:

$ E_d[S, I] = \eta \sum_{i} (I_i - S_i)^2 $


These two terms are combined to get the energy.

$ E[S] = E_p[S] + E_d[S, I] $

Then the posterior of the labeling $S$ given the image $I$ (with temperature parameter $T$) is

$ P(S|I) = \frac{1}{Z} \exp\left( - \frac{E[S]}{T} \right) $

The block of code below initializes the ipython notebook

In [3]:
# Initiialization code
%matplotlib inline
import numpy as np
# from pylab import imshow, show, get_cmap, imread, figure, subplots, title, subplot
import matplotlib.pyplot as plt
from numpy import random
import pylab as pl

The block of code below loads an image and normalizes it to the  range $[-1, 1]$.

In [31]:
im = plt.imread('data/gibbs/gibbs_demo.jpg')
plt.imshow(im)

def myimshow(state):
    plt.imshow(state, interpolation='nearest')

# Preprocess image to range (-1, 1)
def preproc_data(im, scale=0.1, debug=False):
    import skimage.color
    import skimage.transform
    
    tinyim = skimage.transform.rescale(im, scale)
    grayim = skimage.color.rgb2gray(tinyim)

    # Linear map the data to -1, 1
    scale = grayim.max() - grayim.min()
    data = 2 * (grayim - grayim.min()) / scale - 1
    if debug:
        print ('original range: {} {}'.format(grayim.min(), grayim.max()))
        print ('remapped range: {} {}'.format(data.min(), data.max()))

    return [data, tinyim]

[data, im] = preproc_data(im, debug=True) # data is normalized image

original range: 0.0 0.802648263606518
remapped range: -1.0 1.0


The block of code below defines the neighborhood structure for the Gibbs sampler.

In [7]:
def getneighor(y, x, h, w): # get 4-side neighbor
    n = []
    if (x != 0): n.append((y, x-1))
    if (x != w-1): n.append((y, x+1))
    if (y != 0): n.append((y-1, x))
    if (y != h-1): n.append((y+1, x))
    return n

def poslist(h,w):
    '''Get point list of a grid'''
    pos = []
    for x in range(w):
        for y in range(h):
            pos.append((y, x))
    return pos

Define a utility function to compute energy.

In [8]:
def energy_prior(state, gamma):
    total = 0
    (h, w) = state.shape
    pos = poslist(h, w)
    for p in pos:
        neighbor = getneighor(p[0], p[1], h, w) # compute neighbor
        
        for n in neighbor:
            total += state[p[0]][p[1]] * state[n[0]][n[1]]
    E = - gamma * total
    return E
        
def energy_data(state, data, eta):
    E = eta * sum((data - state)**2)
    return E

def energy(state, data, gamma, eta):
    return energy_prior(state, gamma) + energy_data(state, data, eta)

Define the Gibbs sampler.

In [10]:
def gibbs_sampler(state, data, gamma, eta, debug=False): # 0/1 state
    (h, w) = state.shape
    new_state = state.copy()
    pos = poslist(h, w)
    for p in pos:
        neighbor_pos = getneighor(p[0], p[1], h, w)
        neighbor_value = [new_state[n[0]][n[1]] for n in neighbor_pos]

        tmp1 = -gamma * -1 * sum(neighbor_value) # x_i = -1
        tmp2 = -gamma * 1 * sum(neighbor_value) # x_i = 1
        
        # add data term
        v = data[p[0]][p[1]]
        tmp1 += eta * (v - (-1))**2 # x_i = -1
        tmp2 += eta * (v - 1)**2 #  x_i = 1
        
        tmp1 = np.exp(-tmp1)
        tmp2 = np.exp(-tmp2)

        p1 = tmp1 / (tmp1 + tmp2)
        prob = random.uniform() # roll a dice
        
        if (debug): print(p1)
        if (prob > p1):
            new_state[p[0]][p[1]] = 1
        else:
            new_state[p[0]][p[1]] = -1
    return new_state

# Animation: sample with data term included
Run this demo below; make sure to watch the animation as it happens!

In [39]:
%matplotlib qt

from IPython.display import display, clear_output
import statistics
import time
random_seed = 100 # Change this in your experiment
random.seed(random_seed)

(h, w) = data.shape
mat = random.random((h,w))
mat[mat>0.5] = 1
mat[mat<=0.5] = -1
random_state = mat


# Initial the random state
init_state = random_state

# Set parameters
gamma = 100
eta = 1

new_state = random_state.copy()
E = [energy(init_state, data, gamma, eta)]# array of energies at each iteration
A = [statistics.mean(energy(init_state, data, gamma, eta))]

f, ax = plt.subplots() # prepare animation
for i in range(100):
    #clear_output(wait=True)
    new_state = gibbs_sampler(new_state, data, gamma, eta)
    E.append(energy(new_state, data, gamma, eta))
    A.append(statistics.mean(energy(new_state, data, gamma, eta)))
    # time.sleep(1)
    # myimshow(new_state)
    #display(f)

plt.figure(1)
plt.imshow(new_state)


mask = (new_state==1)
fg = data.copy()
x=range(30)

plt.figure(2)
plt.title("Foreground")
plt.imshow(fg, cmap='gray', interpolation='nearest')

plt.show()
print (A)


[5660.947435078384, -740776.3165168162, -876382.2406286609, -905184.3448816666, -918385.215634758, -929585.4677328808, -935985.095683852, -937985.2370526561, -941185.3138604827, -941985.2289741967, -943185.3611662016, -945985.4780382217, -950785.8647339599, -951585.8502047893, -952385.6485635984, -954785.6351788269, -955585.5682004363, -956785.4487198064, -960785.2673362572, -961585.1168439153, -961585.1568785124, -962385.174048492, -962385.0937885848, -962385.2383055758, -964785.1026315305, -967184.8844682152, -967184.9571220573, -967184.9422399678, -967185.0368169068, -967184.9266119128, -967184.9432145348, -967184.9903135018, -967185.0798894753, -967184.903411868, -967185.0518219217, -967184.9367313825, -967185.0031271027, -967185.0033327048, -967185.0996925773, -967184.9838887558, -967185.0401457594, -967184.9355742733, -967185.0209937204, -967185.0263634623, -967184.8813907817, -967184.9659683922, -967185.0109781108, -967185.0415564061, -967184.8792759314, -967184.9831440006, -967

In [14]:
plt.show()


# HW2.2 Gibbs sampler
Set random_seed to a different value (and tell me what it is in your homework!)
1. Try a few different values of $ \gamma $, $ \eta $, including special case that only contains the prior term. What happens when the parameters change? (6 points)
2. Run with different images, plot your results. Find two or three images from the web or your image collection. Can you find an image that causes the model to identify the foreground poorly? Include the image that you use. (4 points)
3. Around what iteration does the sampler converge for the Einstein image with $ \gamma = 20 $ and $ \eta = 1 $ and how do you know it? Don't just say "the image stopped changing very much"! Hint: look carefully at the demo above to see if there's anything that will help you diagnose convergence.(6 points)

In [30]:
from IPython.display import display, clear_output
import time
random_seed = 100 # Change this in your experiment
random.seed(random_seed)


im = plt.imread('data/gibbs/hawk.jpg')
plt.figure(1)
plt.imshow(im)


[data, im] = preproc_data(im, debug=True) # data is normalized image



(h, w) = data.shape
mat = random.random((h,w))
mat[mat>0.5] = 1
mat[mat<=0.5] = -1
random_state = mat


# Initial the random state
init_state = random_state

# Set parameters
gamma = 10
eta = 50

new_state = random_state.copy()
E = [energy(init_state, data, gamma, eta)]# array of energies at each iteration


f, ax = plt.subplots() # prepare animation
for i in range(100):
    new_state = gibbs_sampler(new_state, data, gamma, eta)
    
plt.figure(2)
plt.imshow(new_state)


mask = (new_state==1)
fg = data.copy()
x=range(30)

plt.figure(3)
plt.title("Foreground")
plt.imshow(fg, cmap='gray', interpolation='nearest')

plt.show()

original range: 0.08791852532679682 0.8288677573529414
remapped range: -1.0 1.0


In [17]:
%matplotlib qt

from IPython.display import display, clear_output
import time
random_seed = 100 # Change this in your experiment
random.seed(random_seed)

(h, w) = data.shape
mat = random.random((h,w))
mat[mat>0.5] = 1
mat[mat<=0.5] = -1
random_state = mat


# Initial the random state
init_state = random_state

# Set parameters
gamma = 10
eta = 1

new_state = random_state.copy()
E = [energy(init_state, data, gamma, eta)]# array of energies at each iteration


f, ax = plt.subplots() # prepare animation
for i in range(100):
    new_state = gibbs_sampler(new_state, data, gamma, eta)
    
plt.figure(1)
plt.imshow(new_state)


mask = (new_state==1)
fg = data.copy()
x=range(30)

plt.figure(2)
plt.title("Foreground")
plt.imshow(fg, cmap='gray', interpolation='nearest')












# Initial the random state
init_state = random_state

# Set parameters
gamma = 5
eta = 1

new_state = random_state.copy()
E = [energy(init_state, data, gamma, eta)]# array of energies at each iteration


f, ax = plt.subplots() # prepare animation
for i in range(100):
    new_state = gibbs_sampler(new_state, data, gamma, eta)
    
plt.figure(3)
plt.imshow(new_state)


mask = (new_state==1)
fg = data.copy()
x=range(30)

plt.figure(4)
plt.title("Foreground")
plt.imshow(fg, cmap='gray', interpolation='nearest')









# Initial the random state
init_state = random_state

# Set parameters
gamma = 20
eta = 5

new_state = random_state.copy()
E = [energy(init_state, data, gamma, eta)]# array of energies at each iteration


f, ax = plt.subplots() # prepare animation
for i in range(100):
    new_state = gibbs_sampler(new_state, data, gamma, eta)
    
plt.figure(5)
plt.imshow(new_state)


mask = (new_state==1)
fg = data.copy()
x=range(30)

plt.figure(6)
plt.title("Foreground")
plt.imshow(fg, cmap='gray', interpolation='nearest')














# Initial the random state
init_state = random_state

# Set parameters
gamma = 20
eta = 10

new_state = random_state.copy()
E = [energy(init_state, data, gamma, eta)]# array of energies at each iteration


f, ax = plt.subplots() # prepare animation
for i in range(100):
    new_state = gibbs_sampler(new_state, data, gamma, eta)
    
plt.figure(7)
plt.imshow(new_state)


mask = (new_state==1)
fg = data.copy()
x=range(30)

plt.figure(8)
plt.title("Foreground")
plt.imshow(fg, cmap='gray', interpolation='nearest')












# Initial the random state
init_state = random_state

# Set parameters
gamma = 20
eta = 50

new_state = random_state.copy()
E = [energy(init_state, data, gamma, eta)]# array of energies at each iteration


f, ax = plt.subplots() # prepare animation
for i in range(100):
    new_state = gibbs_sampler(new_state, data, gamma, eta)
    
plt.figure(9)
plt.imshow(new_state)


mask = (new_state==1)
fg = data.copy()
x=range(30)

plt.figure(10)
plt.title("Foreground")
plt.imshow(fg, cmap='gray', interpolation='nearest')









# Initial the random state
init_state = random_state

# Set parameters
gamma = 20
eta = 0.5

new_state = random_state.copy()
E = [energy(init_state, data, gamma, eta)]# array of energies at each iteration


f, ax = plt.subplots() # prepare animation
for i in range(100):
    new_state = gibbs_sampler(new_state, data, gamma, eta)
    
plt.figure(11)
plt.imshow(new_state)


mask = (new_state==1)
fg = data.copy()
x=range(30)

plt.figure(12)
plt.title("Foreground")
plt.imshow(fg, cmap='gray', interpolation='nearest')










# Initial the random state
init_state = random_state

# Set parameters
gamma = 20
eta = 0

new_state = random_state.copy()
E = [energy(init_state, data, gamma, eta)]# array of energies at each iteration


f, ax = plt.subplots() # prepare animation
for i in range(100):
    new_state = gibbs_sampler(new_state, data, gamma, eta)
    
plt.figure(13)
plt.imshow(new_state)


mask = (new_state==1)
fg = data.copy()
x=range(30)



plt.figure(14)
plt.title("Foreground")
plt.imshow(fg, cmap='gray', interpolation='nearest')

plt.show()

In [18]:
# Initial the random state
init_state = random_state

# Set parameters
gamma = 5
eta = 0

new_state = random_state.copy()
E = [energy(init_state, data, gamma, eta)]# array of energies at each iteration


f, ax = plt.subplots() # prepare animation
for i in range(100):
    new_state = gibbs_sampler(new_state, data, gamma, eta)
    
plt.figure(13)
plt.imshow(new_state)


mask = (new_state==1)
fg = data.copy()
x=range(30)



plt.figure(14)
plt.title("Foreground")
plt.imshow(fg, cmap='gray', interpolation='nearest')

plt.show()